- https://ai.plainenglish.io/understanding-large-language-model-based-agents-27bee5c82cec

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## LCEL (LangChain Expression Language)

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-3.5-turbo")
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

chain.invoke("ice cream")

'Why did the ice cream truck break down?\n\nBecause it had too many sundaes!'

## Tool uses